In [ ]:

import re
import pandas as pd
from nltk import tokenize
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize as wt
import nltk
import string
#nltk.download('averaged_perceptron_tagger')
#nltk.download('wordnet')
from nltk.corpus import wordnet
import ast
from nltk.tokenize import word_tokenize as wt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#Reading Data

hgrd=pd.read_excel(r'C:\Users\rramn\Documents\Healthcare_analytics\physician_reviews_mining\healthgrades_latest.xlsx')
ratemd=pd.read_excel(r'C:\Users\rramn\Documents\Healthcare_analytics\physician_reviews_mining\Reviews\ratemds_reviews.xlsx')

In [ ]:
#Pre processing - sentence split
sentrm = []
senthg=[]
nr,nc=hgrd.shape
nr1, nc1 = ratemd.shape
# hgrd.info()

for j in range(nr1):
    t1 = ratemd.iloc[j, 2]
    #sent1=re.split("[" + string.punctuation + "]+", t1)
    sent1 = re.split(r'[\,\.\?!][\'"\)\]]*|and|but', t1)
    sentrm.append(sent1)
for i in range(nr):
    t=hgrd.iloc[i,7]
    #sent=re.split("[" + string.punctuation + "]+", t)
    sent=re.split(r'[\,\.\?!][\'"\)\]]*|and|but',t)
    senthg.append(sent)


hgrd['Review_cleaned']=senthg
ratemd['Review_cleaned']=sentrm

In [ ]:
#VADER Sentiment analysis
l=[]
m=[]
n=[]
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser1 = SentimentIntensityAnalyzer()
def sentiment_analyzer_scores(sentence):
   score = analyser1.polarity_scores(sentence)
   #print("{:-<40} {}".format(sentence, str(score)))
   #print(score)
   if score['compound']>0:
       l.append(1)
       return "Positive",score['compound'] 
   elif score['compound']<0:       
       m.append(1)
       return "Negative",score['compound']    
   else:       
       n.append(1)
       return "Neutral",score['compound']

In [ ]:
#Getting sentiment scores for Healthgrades
def sentence_sentiment(df,index=None):
    senti_list = []
    nr1,nc1 = df.shape
    cum_score=[]
    name_list=[]
    for j in range(nr1):
        t2=df.iloc[j,index]
        sub_list = []
        sub_list1=[]
        c=0
        nam=df.iloc[j,0]
        for x in range(len(t2)):
            d = {}
            e = {}
            f= {}
            senti,score = sentiment_analyzer_scores(t2[x])
            #print(senti,score)
            c=c+score
            d[t2[x]] = senti
            e[t2[x]] = score
            f[t2[x]] = nam
            senti_list.append(d)
            cum_score.append(e)
            name_list.append(f)
            #sub_list.append(d)
            #sub_list1.append(e)
            # for k, v in sub_list:
            #     if v == 'Neutral':
            #         del sub_list[k]
        
    return senti_list,cum_score,name_list

sentiment,cu_score,dname = sentence_sentiment(hgrd,7)
print(len(sentiment))
print(len(cu_score))
print(len(dname))
print(len(l))
print(len(m))
print(len(n))

In [ ]:
#Getting sentiment scores for Ratemds
def sentence_sentiment(df,index=None):
    senti_list = []
    nr1,nc1 = df.shape
    cum_score=[]
    name_list=[]
    for j in range(nr1):
        t2=df.iloc[j,index]
        sub_list = []
        sub_list1=[]
        c=0
        nam=df.iloc[j,0]
        for x in range(len(t2)):
            d = {}
            e = {}
            f= {}
            senti,score = sentiment_analyzer_scores(t2[x])
            #print(senti,score)
            c=c+score
            d[t2[x]] = senti
            e[t2[x]] = score
            f[t2[x]] = nam
            senti_list.append(d)
            cum_score.append(e)
            name_list.append(f)
            #sub_list.append(d)
            #sub_list1.append(e)
            # for k, v in sub_list:
            #     if v == 'Neutral':
            #         del sub_list[k]
        
    return senti_list,cum_score,name_list

sentiment,cu_score,dname = sentence_sentiment(ratemd,3)
print(len(sentiment))
print(len(cu_score))
print(len(dname))
print(len(l))
print(len(m))
print(len(n))

In [ ]:
#Merging all the results into single dataset for Regression Analysis - Healthgrades
hgrd1=pd.read_excel(r'C:\Users\rramn\Documents\Healthcare_analytics\physician_reviews_mining\Reviews\healthgrades1.xlsx',sheet_name="Sheet6")
hgrd2=pd.read_excel(r'C:\Users\rramn\Documents\Healthcare_analytics\physician_reviews_mining\Reviews\healthgrades1.xlsx',sheet_name="Sheet5")
hgrd3=pd.read_excel(r'C:\Users\rramn\Documents\Healthcare_analytics\physician_reviews_mining\Reviews\healthgrades1.xlsx',sheet_name="Sheet4")

tem=pd.merge(hgrd1,hgrd2,on=["DoctorName"],how='left')
tem1=pd.merge(tem,hgrd3,on=["DoctorName"],how='left')

tem1=tem1.fillna(0)
np.sum(tem1.isna())
hgrd4=pd.read_excel(r'C:\Users\rramn\Documents\Healthcare_analytics\physician_reviews_mining\Healthgrade_topic_result.xlsx',sheet_name="Sheet6")

tem2=pd.merge(tem1,hgrd4,on=["DoctorName"],how='left')

tem2=tem2.fillna(0)
hgrd5=pd.read_excel(r'C:\Users\rramn\Documents\Healthcare_analytics\physician_reviews_mining\Reviews\healthgrades2.xlsx')

tem3=pd.merge(hgrd5,tem2,on=["DoctorName"],how='left')

tem3.to_excel(r'C:\Users\rramn\Documents\Healthcare_analytics\physician_reviews_mining\Reviews\healthgradesfinal.xlsx',header=True)

In [ ]:
#Merging all the results into single dataset for Regression Analysis - Ratemds

ratemd1=pd.read_excel(r'C:\Users\rramn\Documents\Healthcare_analytics\physician_reviews_mining\Reviews\ratemds1.xlsx',sheet_name="Sheet9")
ratemd2=pd.read_excel(r'C:\Users\rramn\Documents\Healthcare_analytics\physician_reviews_mining\Reviews\ratemds1.xlsx',sheet_name="Sheet10")
ratemd3=pd.read_excel(r'C:\Users\rramn\Documents\Healthcare_analytics\physician_reviews_mining\Reviews\ratemds1.xlsx',sheet_name="Sheet11")

rem=pd.merge(ratemd1,ratemd2,on=["Doctor_names"],how='left')

rem1=pd.merge(rem,ratemd3,on=["Doctor_names"],how='left')

ratemd4=pd.read_excel(r'C:\Users\rramn\Documents\Healthcare_analytics\physician_reviews_mining\Ratemd_topic_result.xlsx',sheet_name="Sheet5")       

rem2=pd.merge(rem1,ratemd4,on=["Doctor_names"],how='left')

rem2=rem2.fillna(0)
ratemd5=pd.read_excel(r'C:\Users\rramn\Documents\Healthcare_analytics\physician_reviews_mining\ratemdswithrating.xlsx')
ratemd5.info()

rem3=pd.merge(ratemd5,rem2,on=["Doctor_names"],how='left')
